# Block Ciphers

In [ ]:
from aes import AES

from diffusion_confusion import flip_bit, hamming_distance
from diffusion_confusion import aes_diffusion, aes_confusion

## AES

In [2]:
key = bytes.fromhex("2b7e151628aed2a6abf7158809cf4f3c")
plaintext = bytes.fromhex("3243f6a8885a308d313198a2e0370734")

aes = AES(key)
round_keys = aes.key_expansion()

# Round key 0
print("Round Key 0:")
for row in round_keys[0]:
    print(" ".join(f"{b:02x}" for b in row))



print("\nRound Key 1:")
for row in round_keys[1]:
    print(" ".join(f"{b:02x}" for b in row))


Round Key 0:
2b 28 ab 09
7e ae f7 cf
15 d2 15 4f
16 a6 88 3c

Round Key 1:
a0 88 23 2a
fa 54 a3 6c
fe 2c 39 76
17 b1 39 05


In [3]:
state = [
    [0x01, 0x02, 0x03, 0x04],
    [0x05, 0x06, 0x07, 0x08],
    [0x09, 0x0a, 0x0b, 0x0c],
    [0x0d, 0x0e, 0x0f, 0x10],
]

round_key = [
    [0x10, 0x0f, 0x0e, 0x0d],
    [0x0c, 0x0b, 0x0a, 0x09],
    [0x08, 0x07, 0x06, 0x05],
    [0x04, 0x03, 0x02, 0x01],
]

result = AES.add_round_key(state, round_key)

for row in result:
    print(" ".join(f"{b:02x}" for b in row))


11 0d 0d 09
09 0d 0d 01
01 0d 0d 09
09 0d 0d 11


In [4]:
state = [
    [0x00, 0x01, 0x02, 0x03],
    [0x04, 0x05, 0x06, 0x07],
    [0x08, 0x09, 0x0a, 0x0b],
    [0x0c, 0x0d, 0x0e, 0x0f],
]

result = AES.byte_substitution(state)

for row in result:
    print(" ".join(f"{b:02x}" for b in row))


63 7c 77 7b
f2 6b 6f c5
30 01 67 2b
fe d7 ab 76


In [5]:
state = [
    [0x00, 0x01, 0x02, 0x03],
    [0x10, 0x11, 0x12, 0x13],
    [0x20, 0x21, 0x22, 0x23],
    [0x30, 0x31, 0x32, 0x33],
]

result = AES.shift_rows(state)

for row in result:
    print(" ".join(f"{b:02x}" for b in row))


00 01 02 03
11 12 13 10
22 23 20 21
33 30 31 32


In [6]:
state = [
    [0xdb, 0xf2, 0x01, 0xc6],
    [0x13, 0x0a, 0x01, 0xc6],
    [0x53, 0x22, 0x01, 0xc6],
    [0x45, 0x5c, 0x01, 0xc6],
]

result = AES.mix_column(state)

for row in result:
    print(" ".join(f"{b:02x}" for b in row))


8e 9f 01 c6
4d dc 01 c6
a1 58 01 c6
bc 9d 01 c6


In [7]:
state = [
    [0xdb, 0xf2, 0x01, 0xc6],
    [0x13, 0x0a, 0x01, 0xc6],
    [0x53, 0x22, 0x01, 0xc6],
    [0x45, 0x5c, 0x01, 0xc6]
]

expected = [
    [0x8e, 0x9f, 0x01, 0xc6],
    [0x4d, 0xdc, 0x01, 0xc6],
    [0xa1, 0x58, 0x01, 0xc6],
    [0xbc, 0x9d, 0x01, 0xc6]
]

result = AES.mix_column(state)

# Print results nicely
for r_row, e_row in zip(result, expected):
    print("Result:   ", " ".join(f"{b:02x}" for b in r_row))
    print("Expected: ", " ".join(f"{b:02x}" for b in e_row))
    print("Match?   ", r_row == e_row)
    print()


Result:    8e 9f 01 c6
Expected:  8e 9f 01 c6
Match?    True

Result:    4d dc 01 c6
Expected:  4d dc 01 c6
Match?    True

Result:    a1 58 01 c6
Expected:  a1 58 01 c6
Match?    True

Result:    bc 9d 01 c6
Expected:  bc 9d 01 c6
Match?    True



In [8]:
b = bytes(range(16))
m = AES.to_matrix(b)
print("Matrix:")
for row in m:
    print(" ".join(f"{x:02x}" for x in row))

print("\nBack to bytes:")
print(AES.to_bytes(m).hex())

print("\nOriginal:")
print(b.hex())

assert AES.to_bytes(m) == b

Matrix:
00 04 08 0c
01 05 09 0d
02 06 0a 0e
03 07 0b 0f

Back to bytes:
000102030405060708090a0b0c0d0e0f

Original:
000102030405060708090a0b0c0d0e0f


In [15]:
key = bytes.fromhex("2b7e151628aed2a6abf7158809cf4f3c")
plaintext = bytes.fromhex("3243f6a8885a308d313198a2e0370734")

aes = AES(key)
ciphertext = aes.encrypt(plaintext)

print("Ciphertext:", ciphertext.hex())
print("Expected:  ", "3925841d02dc09fbdc118597196a0b32")
print("✅ Match?  ", ciphertext.hex() == "3925841d02dc09fbdc118597196a0b32")

Ciphertext: 3925841d02dc09fbdc118597196a0b32
Expected:   3925841d02dc09fbdc118597196a0b32
✅ Match?   True


In [10]:
"""key = bytes.fromhex("000102030405060708090a0b0c0d0e0f")
plaintext = bytes.fromhex("00112233445566778899aabbccddeeff")
expected_ciphertext = "69c4e0d86a7b0430d8cdb78070b4c55a"""

"""key = bytes.fromhex("00000000000000000000000000000000")
plaintext = bytes.fromhex("00000000000000000000000000000000")
expected_ciphertext = "66e94bd4ef8a2c3b884cfa59ca342b2e"""

key = bytes.fromhex("ffffffffffffffffffffffffffffffff")
plaintext = bytes.fromhex("ffffffffffffffffffffffffffffffff")
expected_ciphertext = "a1f6258c877d5fcd8964484538bfc92c"

"""key = bytes.fromhex("2b7e151628aed2a6abf7158809cf4f3c")
plaintext = bytes.fromhex("3243f6a8885a308d313198a2e0370734")
expected_ciphertext = "3925841d02dc09fbdc118597196a0b32"""

aes = AES(key)
ciphertext = aes.encrypt(plaintext)

print("Result:   ", ciphertext.hex())
print("Expected: ", expected_ciphertext)
print("Match?    ", ciphertext.hex() == expected_ciphertext)

Result:    bcbf217cb280cf30b2517052193ab979
Expected:  a1f6258c877d5fcd8964484538bfc92c
Match?     False


In [21]:
for round_idx, matrix in enumerate(aes.key_expansion()):
    print(f"Round {round_idx}:")
    for row in matrix:
        print(" ".join(f"{b:02x}" for b in row))


Round 0:
ff ff ff ff
ff ff ff ff
ff ff ff ff
ff ff ff ff
Round 1:
e8 17 e8 17
e9 16 e9 16
e9 16 e9 16
e9 16 e9 16
Round 2:
ad ba 52 45
ae b8 51 47
ae b8 51 47
19 0f e6 f0
Round 3:
09 b3 e1 a4
0e b6 e7 a0
22 9a cb 8c
77 78 9e 6e
Round 4:
e1 52 b3 17
6a dc 3b 9b
bd 27 ec 60
3e 46 d8 b6
Round 5:
e5 b7 04 13
ba 66 5d c6
f3 d4 38 58
ce 88 50 e6
Round 6:
71 c6 c2 d1
d0 b6 eb 2d
7d a9 91 c9
b3 3b 6b 8d
Round 7:
e9 2f ed 3c
0d bb 50 7d
20 89 18 d1
8d b6 dd 50
Round 8:
96 b9 54 68
33 88 d8 a5
73 fa e2 33
66 d0 0d 5d
Round 9:
8b 32 66 0e
f0 78 a0 05
3f c5 27 14
23 f3 fe a3
Round 10:
d6 e4 82 8c
0a 72 d2 d7
35 f0 d7 c3
88 7b 85 26


In [4]:
input_bytes = bytes(range(16))
aes = AES(key)
AES.to_bytes(AES.to_matrix(input_bytes))

b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f'

### Add Round Key

### Substitution Bytes

### Shift Rows

### Mix Columns

### Key Expansion

### Encryption

## AES Diffusion and Confusion

In [17]:
import matplotlib.pyplot as plt

def run_monte_carlo(function, num_rounds=None, num_trials=1000):
    distances = []
    for i in range(num_trials):
        distance = function(num_rounds)
        distances.append(distance)
    return distances

In [ ]:
diffusion_results = run_monte_carlo(aes_diffusion, num_rounds=1, num_trials=1000)
confusion_results = run_monte_carlo(aes_confusion, num_rounds=10, num_trials=1000)

## Conclusion